In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from glob import glob
import keras
from keras.applications.vgg16 import preprocess_input,VGG16
from keras.models import Sequential,Model
from keras.layers import MaxPooling2D,Conv2D,Dense,BatchNormalization,Dropout,GlobalAveragePooling2D,Flatten,Input, Activation
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.utils.vis_utils import plot_model
import warnings
warnings.filterwarnings('ignore')

# Connecting to Google Drive -- DATASET ACCESS
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# Accessing Dataset from Google Drive
Training = '/content/gdrive/My Drive/Colab Notebooks/SET - Training'
Testing = '/content/gdrive/My Drive/Colab Notebooks/SET - Testing'

# Creating Tumor Class Labels
labels = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [ ]:
# Creating Training and Testing Arrays
X_train = []
y_train = []

# Image Size
image_size = 224

# Transferring Images from Google Drive to Arrays
for i in labels:
    folderPath = os.path.join(r'C:\Users\adamr\Downloads\archive (6)\SET - Training', Training, i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
        
for i in labels:
    folderPath = os.path.join(r'C:\Users\adamr\Downloads\archive (6)\SET - Testing', Testing, i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)
        
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
# DATA AUGMENTATION
# Rescaling used in all trials, including control
# Rescale protocols are shown in other files!

rescale = ImageDataGenerator(
    rescale = 1./255,
)

In [ ]:
# Shuffling Images from Training and Testing Arrays
X_train, y_train = shuffle(X_train,y_train, random_state = 101)

# Splitting Arrays to 80% Training, 20% Testing
# X_train and X_test contain the MRI images, y_train and y_test contain the labels for the respective images
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state = 101)

# Applying the above rescales + augmentations
rescale.fit(X_train)

In [ ]:
# Creating new y_train array with numerical labels for images
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

# Creating new y_test array with numerical labels for images
y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
# Custom CNN Model Architecture
# Input Image Size 
INPUT_SHAPE = (224, 224, 3)

model = Sequential()

## FEATURE LEARNING BLOCK ##
# First Convolutional Layer -- Features: 128, Kernel Size: (3,3), Strides: (1,1), Padding: Valid, Activation Function: RELU, Batch Normalization
model.add(Conv2D(128, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Second Convolutional Layer -- Features: 64, Kernel Size: (3,3), Strides: (1,1), Padding: Valid, Activation Function: RELU, Batch Normalization
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Third Convolutional Layer -- Features: 32, Kernel Size: (3,3), Strides: (1,1), Padding: Valid, Activation Function: RELU, Batch Normalization
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
##################################

## CONCATONATED DENSE LAYERS BLOCK ##
# Flattening Layer
model.add(Flatten())

# First Dense Layer -- Activation Function: RELU, Dropout: 0.2
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

# Second Dense Layer -- Activation Function: RELU
model.add(Dense(64, activation='relu'))

# Third Dense Layer -- Activation Function: Softmax -- Returns Numerical Output (0-3) to Classification Block
model.add(Dense(4, activation='softmax'))
##################################

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Running Model -- Epochs: 20, Steps Per Epoch: 69, Validation Split: 0.1, Batch Size: 32
history = model.fit(X_train, y_train, steps_per_epoch = 69, validation_split=0.1, epochs=20, batch_size=32)

In [ ]:
# Figure Color Hex Codes
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']
colors_red = ["#331313", "#582626", '#9E1717', '#D35151', '#E9B4B4']
colors_green = ['#01411C','#4B6F44','#4F7942','#74C365','#D0F0C0']

# Training Accuracy vs Validation Accuracy and Training Loss vs Validation Loss
filterwarnings('ignore')

epochs = [i for i in range(20)]
fig, ax = plt.subplots(1,2,figsize=(14,7))
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

fig.text(s='Epochs vs. Training and Validation Accuracy/Loss',size=18,fontweight='bold',
             fontname='monospace',color=colors_dark[1],y=1,x=0.28,alpha=0.8)

sns.despine()
ax[0].plot(epochs, train_acc,color=colors_green[3],
           label = 'Training Accuracy')
ax[0].plot(epochs, val_acc,color=colors_red[3],
           label = 'Validation Accuracy')
ax[0].legend(frameon=False)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')

sns.despine()
ax[1].plot(epochs, train_loss, color=colors_green[3],
           label ='Training Loss')
ax[1].plot(epochs, val_loss, color=colors_red[3],
           label = 'Validation Loss')
ax[1].legend(frameon=False)
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Training & Validation Loss')

fig.show()


# Evaluating CNN Model on Test Sets
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

# Returns Accuracy, Precision, Recall, F1 Score Report
print(classification_report(y_test_new,pred))

# Generating Confusion Matricx
fig,ax=plt.subplots(1,1,figsize=(14,7))
sns.heatmap(confusion_matrix(y_test_new,pred),ax=ax,xticklabels=labels,yticklabels=labels,annot=True,
           cmap=colors_green[::-1],alpha=0.7,linewidths=2,linecolor=colors_dark[3])
fig.text(s='Heatmap of the Confusion Matrix',size=18,fontweight='bold',
             fontname='monospace',color=colors_dark[1],y=0.92,x=0.28,alpha=0.8)

plt.show()